# Creating a simple Auto-encoders from scratch with Fashion-MNIST dataset.

## 1) Import modules

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.datasets import mnist
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.regularizers import l1
from tensorflow.keras.optimizers import Adam

In [ ]:
import tensorflow as tf
tf.__version__

## 2) Utility Function

In [ ]:
def plot_autoencoder_outputs(autoencoder, n, dims):

    n = 5
    plt.figure(figsize=(10, 4.5))
    decoded_imgs = autoencoder.predict(x_test)
    
    for i in range(n):
        
        # plot original image
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(x_test[i].reshape(*dims))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        if i == n/2:
            ax.set_title('Original Images')

        # plot reconstruction 
        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(decoded_imgs[i].reshape(*dims))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        if i == n/2:
            ax.set_title('Reconstructed Images')
            
    plt.show()

## 3) Loading and preparing the dataset

In [ ]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

print(x_train.shape)
print(x_test.shape)

## 4) Building the Auto-Encoder

In [ ]:
input_size = 784

n_neurons = 64

import tensorflow as tf

print('tf version', tf.__version__)

input_img = Input(shape=(input_size,))

code = Dense(n_neurons, activation='relu')(input_img)

output_img = Dense(input_size, activation='sigmoid')(code)

autoencoder = Model(input_img, output_img)

autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder.fit(x_train, x_train, epochs=5)



## 5) Visualize the results Original vs Reconstructed Images

In [ ]:
plot_autoencoder_outputs(autoencoder, 5, (28, 28))

In [ ]:
weights = autoencoder.get_weights()[0].T

n = 10
plt.figure(figsize=(20, 5))
for i in range(n):
    ax = plt.subplot(1, n, i + 1)
    plt.imshow(weights[i+0].reshape(28, 28))
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    

# Deep Auto-Encoder

## 4) Buidling the Deep Auto-Encoder

In [ ]:
input_size = 784

hidden_size = 128

code_size = 128

input_img = Input(shape=(input_size,))

hidden_1 = Dense(hidden_size, activation='relu')(input_img)

code = Dense(code_size, activation='relu')(hidden_1)

hidden_2 = Dense(hidden_size, activation='relu')(code)

output_img = Dense(input_size, activation='sigmoid')(hidden_2)

autoencoder = Model(input_img, output_img)

autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder.fit(x_train, x_train, epochs=3)

## 5) Visualize the results Original vs Reconstructed Images

In [ ]:
plot_autoencoder_outputs(autoencoder, 5, (28, 28))

# Denoising Autoencoder

## 1) Generating Noisy Images

In [ ]:
noise_factor = 0.4
x_train_noisy = x_train + noise_factor * np.random.normal(size=x_train.shape) 
x_test_noisy = x_test + noise_factor * np.random.normal(size=x_test.shape)

x_train_noisy = np.clip(x_train_noisy, 0.0, 1.0)
x_test_noisy = np.clip(x_test_noisy, 0.0, 1.0)

n = 5
plt.figure(figsize=(10, 4.5))
for i in range(n):
    # plot original image
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if i == n/2:
        ax.set_title('Original Images')

    # plot noisy image 
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(x_test_noisy[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if i == n/2:
        ax.set_title('Noisy Input')

## 2) Buidling the Deep Auto-Encoder for Image Denoising

In [ ]:
input_size = 784

hidden_size = 128

code_size = 32

input_img = Input(shape=(input_size,))

hidden_1 = Dense(hidden_size, activation='relu')(input_img)

code = Dense(code_size, activation='relu')(hidden_1)

hidden_2 = Dense(hidden_size, activation='relu')(code)

output_img = Dense(input_size, activation='sigmoid')(hidden_2)

autoencoder = Model(input_img, output_img)

autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder.fit(x_train_noisy, x_train, epochs=10)

## 3) Visualize the results Original vs Reconstructed Images

In [ ]:
n = 5
plt.figure(figsize=(10, 7))

images = autoencoder.predict(x_test_noisy)

for i in range(n):
    # plot original image
    ax = plt.subplot(3, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if i == n/2:
        ax.set_title('Original Images')

    # plot noisy image 
    ax = plt.subplot(3, n, i + 1 + n)
    plt.imshow(x_test_noisy[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if i == n/2:
        ax.set_title('Noisy Input')
        
    # plot noisy image 
    ax = plt.subplot(3, n, i + 1 + 2*n)
    plt.imshow(images[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if i == n/2:
        ax.set_title('Autoencoder Output')